In [1]:
import os, pickle, warnings, sys
from time import perf_counter
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

warnings.filterwarnings("ignore")

In [2]:
SEED = 42
USE_DEMO_DATA = True   # ← لو لديك X_train... غيّرها إلى False واستخدم قسم "YOUR DATA" بالأسفل
SAVE_DIR = "./outputs"
os.makedirs(SAVE_DIR, exist_ok=True)


In [3]:
def load_demo_data(test_size=0.2, random_state=SEED):
    from sklearn.datasets import load_breast_cancer
    from sklearn.model_selection import train_test_split
    X, y = load_breast_cancer(return_X_y=True, as_frame=True)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    return X_train, X_test, y_train, y_test


In [4]:
def load_your_data():
    # مثال (عدّل حسب بياناتك):
    # return X_train, X_test, y_train, y_test
    return None

if USE_DEMO_DATA:
    X_train, X_test, y_train, y_test = load_demo_data()
else:
    maybe = load_your_data()
    if maybe is None:
        raise ValueError("رجاءً زوّد X_train, X_test, y_train, y_test في load_your_data().")


In [5]:
def build_lr_pipeline():
    return Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=1000, solver="lbfgs", random_state=SEED))
    ])

def build_xgb_pipeline():
    try:
        from xgboost import XGBClassifier
    except Exception as e:
        print("⚠️ XGBoost غير مثبت. ثبّت الحزمة ثم أعد التشغيل: pip install xgboost")
        return None
    return Pipeline([
        ("scaler", StandardScaler()),   # ثابت للتوحيد مع الـ MLOps
        ("clf", XGBClassifier(
            n_estimators=100,
            max_depth=4,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_lambda=1.0,
            tree_method="hist",  # أسرع/أخف عادة
            n_jobs=1,
            random_state=SEED,
            eval_metric="logloss",
        ))
    ])


In [6]:
def train_and_eval(pipe, name, X_train, y_train, X_test, y_test):
    t0 = perf_counter()
    pipe.fit(X_train, y_train)
    train_time_s = perf_counter() - t0

    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")

    return {
        "Model": name,
        "Accuracy": acc,
        "F1": f1,
        "Train_Time_s": train_time_s
    }

In [7]:
def measure_model_size(pipe, out_path):
    b = pickle.dumps(pipe, protocol=pickle.HIGHEST_PROTOCOL)
    with open(out_path, "wb") as f:
        f.write(b)
    size_kb = len(b) / 1024.0
    return size_kb


In [8]:
def measure_inference_time(pipe, X_test):
    t0 = perf_counter()
    _ = pipe.predict(X_test)
    total_s = perf_counter() - t0
    single_ms = (total_s / len(X_test)) * 1000.0
    return total_s, single_ms


In [9]:
results_metrics = []
results_sizes = []
results_timing = []


In [10]:
lr_pipe = build_lr_pipeline()
lr_metrics = train_and_eval(lr_pipe, "LR-Pipeline", X_train, y_train, X_test, y_test)
lr_size_kb = measure_model_size(lr_pipe, os.path.join(SAVE_DIR, "lr_pipeline.pkl"))
lr_inf_total_s, lr_inf_single_ms = measure_inference_time(lr_pipe, X_test)

results_metrics.append(lr_metrics)
results_sizes.append({"Model": "LR-Pipeline", "Size_KB": lr_size_kb})
results_timing.append({
    "Model": "LR-Pipeline",
    "Total_Test_Inference_Time_s": lr_inf_total_s,
    "Single_Inference_Time_ms": lr_inf_single_ms
})

In [11]:
xgb_pipe = build_xgb_pipeline()
if xgb_pipe is not None:
    xgb_metrics = train_and_eval(xgb_pipe, "XGB-Pipeline", X_train, y_train, X_test, y_test)
    xgb_size_kb = measure_model_size(xgb_pipe, os.path.join(SAVE_DIR, "xgb_pipeline.pkl"))
    xgb_inf_total_s, xgb_inf_single_ms = measure_inference_time(xgb_pipe, X_test)

    results_metrics.append(xgb_metrics)
    results_sizes.append({"Model": "XGB-Pipeline", "Size_KB": xgb_size_kb})
    results_timing.append({
        "Model": "XGB-Pipeline",
        "Total_Test_Inference_Time_s": xgb_inf_total_s,
        "Single_Inference_Time_ms": xgb_inf_single_ms
    })


In [12]:
metrics_df = pd.DataFrame(results_metrics).set_index("Model")
sizes_df   = pd.DataFrame(results_sizes).set_index("Model")
timing_df  = pd.DataFrame(results_timing).set_index("Model")

metrics_path = os.path.join(SAVE_DIR, "metrics.csv")
sizes_path   = os.path.join(SAVE_DIR, "model_sizes.csv")
timing_path  = os.path.join(SAVE_DIR, "inference_times.csv")

metrics_df.to_csv(metrics_path)
sizes_df.to_csv(sizes_path)
timing_df.to_csv(timing_path)

print("\n===== Task 1: Scores =====")
print(metrics_df.round(4))
print("\n===== Task 2.1: Model Sizes (KB) =====")
print(sizes_df.round(2))
print("\n===== Task 2.2: Inference Times =====")
print(timing_df.round(6))


===== Task 1: Scores =====
              Accuracy      F1  Train_Time_s
Model                                       
LR-Pipeline     0.9825  0.9825        0.0730
XGB-Pipeline    0.9561  0.9558        0.1675

===== Task 2.1: Model Sizes (KB) =====
              Size_KB
Model                
LR-Pipeline      2.61
XGB-Pipeline   104.52

===== Task 2.2: Inference Times =====
              Total_Test_Inference_Time_s  Single_Inference_Time_ms
Model                                                              
LR-Pipeline                      0.012247                  0.107430
XGB-Pipeline                     0.009566                  0.083915


# Task 2.3: ESP32 Analysis (auto paragraph)

In [31]:
def esp32_analysis_text(sizes_df, timing_df):
    # مواصفات تقريبية: RAM ~ 520 KB، Flash بعدة ميغابايت (حسب اللوحة)
    SRAM_KB = 520.0
    ONE_SEC_BUDGET_MS = 1000.0  # مثال: استدلال كل ثانية

    def row_or_none(df, name):
        return df.loc[name] if name in df.index else None

    lr_s = row_or_none(sizes_df, "LR-Pipeline")
    xg_s = row_or_none(sizes_df, "XGB-Pipeline")
    lr_t = row_or_none(timing_df, "LR-Pipeline")
    xg_t = row_or_none(timing_df, "XGB-Pipeline")

    lines = []
    lines.append("تحليل الجاهزية للنشر على ESP32:\n")

    # الذاكرة
    lines.append("1) الملاءمة الذاكرية:")
    if lr_s is not None:
        lines.append(f"   • LR: حجم النموذج ≈ {lr_s['Size_KB']:.1f} KB (يُخزَّن عادةً في الـFlash ويُحمَّل جزئيًا/كاملًا إلى الـSRAM وقت الاستدلال).")
    if xg_s is not None:
        lines.append(f"   • XGB: حجم النموذج ≈ {xg_s['Size_KB']:.1f} KB.")
    lines.append(f"   - قياس مرجعي: ESP32 يملك نحو {SRAM_KB:.0f} KB SRAM إجمالي (والحيّز المتاح للتطبيق أقل من ذلك).")
    if (lr_s is not None) and (xg_s is not None):
        more_constrained = "XGB-Pipeline" if xg_s['Size_KB'] > lr_s['Size_KB'] else "LR-Pipeline"
        lines.append(f"   ⇒ الأكثر تقييدًا من ناحية الحجم هو: {more_constrained}.")
    lines.append("")

    # الزمن
    lines.append("2) الكفاءة الزمنية (استدلال آنٍ كل 1 ثانية كمثال):")
    if lr_t is not None:
        lines.append(f"   • LR: زمن الاستدلال المفرد ≈ {lr_t['Single_Inference_Time_ms']:.3f} ms.")
    if xg_t is not None:
        lines.append(f"   • XGB: زمن الاستدلال المفرد ≈ {xg_t['Single_Inference_Time_ms']:.3f} ms.")
    lines.append(f"   - بميزانية ~{ONE_SEC_BUDGET_MS:.0f} ms/ث، كلاهما عادةً مقبول إذا كان الزمن بالملّي ثوانٍ أحادية أو عشرات قليلة.")
    lines.append("")

    # الخلاصة + تحسينات
    choice = None
    if (lr_s is not None) and (xg_s is not None) and (lr_t is not None) and (xg_t is not None):
        # قرار بسيط: اختر الأصغر والأسرع
        score_lr  = lr_s['Size_KB'] * 0.5 + lr_t['Single_Inference_Time_ms'] * 0.5
        score_xgb = xg_s['Size_KB'] * 0.5 + xg_t['Single_Inference_Time_ms'] * 0.5
        choice = "LR-Pipeline" if score_lr <= score_xgb else "XGB-Pipeline"
    elif lr_s is not None and lr_t is not None:
        choice = "LR-Pipeline"

    lines.append("3) الخلاصة:")
    if choice is not None:
        lines.append(f"   • الخيار الأنسب على أساس الكفاءة وقيود ESP32: **{choice}**.")
    else:
        lines.append("   • تعذّر الحسم لغياب قياسات أحد النموذجين.")

    lines.append("   • لو احتجت لنشر النموذج الأقل كفاءة: استخدم تقنيات التحسين مثل: تقليل الميزات، التكميم 8-bit أو fixed-point، تقليل عدد الأشجار/العمق (لـXGB)، التحويل إلى C/MCU عبر m2cgen أو Treelite، واستخدام استدلال أحادي الخيط، مع إدارة ذاكرة صارمة.")
    return "\n".join(lines)

In [32]:
print("\n===== Task 2.3: ESP32 Analysis =====")
print(esp32_analysis_text(sizes_df, timing_df))

print(f"\n📁 Saved to: {os.path.abspath(SAVE_DIR)}")
print(" - metrics.csv (الدقة و F1 وزمن التدريب)")
print(" - model_sizes.csv (أحجام النماذج بالـKB)")
print(" - inference_times.csv (زمن الاستدلال الكلّي والمفرد)")
print(" - lr_pipeline.pkl / xgb_pipeline.pkl")


===== Task 2.3: ESP32 Analysis =====
تحليل الجاهزية للنشر على ESP32:

1) الملاءمة الذاكرية:
   • LR: حجم النموذج ≈ 2.6 KB (يُخزَّن عادةً في الـFlash ويُحمَّل جزئيًا/كاملًا إلى الـSRAM وقت الاستدلال).
   • XGB: حجم النموذج ≈ 104.5 KB.
   - قياس مرجعي: ESP32 يملك نحو 520 KB SRAM إجمالي (والحيّز المتاح للتطبيق أقل من ذلك).
   ⇒ الأكثر تقييدًا من ناحية الحجم هو: XGB-Pipeline.

2) الكفاءة الزمنية (استدلال آنٍ كل 1 ثانية كمثال):
   • LR: زمن الاستدلال المفرد ≈ 0.107 ms.
   • XGB: زمن الاستدلال المفرد ≈ 0.084 ms.
   - بميزانية ~1000 ms/ث، كلاهما عادةً مقبول إذا كان الزمن بالملّي ثوانٍ أحادية أو عشرات قليلة.

3) الخلاصة:
   • الخيار الأنسب على أساس الكفاءة وقيود ESP32: **LR-Pipeline**.
   • لو احتجت لنشر النموذج الأقل كفاءة: استخدم تقنيات التحسين مثل: تقليل الميزات، التكميم 8-bit أو fixed-point، تقليل عدد الأشجار/العمق (لـXGB)، التحويل إلى C/MCU عبر m2cgen أو Treelite، واستخدام استدلال أحادي الخيط، مع إدارة ذاكرة صارمة.

📁 Saved to: /content/outputs
 - metrics.csv (الدقة و F1 وزمن التدريب)
 -